# Project Overview:

**Goal:** Build an encoder-decoder LSTM model to predict stock prices 30 days ahead using historical data.
**Required Libraries:**

    - Pandas , NumPy, Matplotlib, Seaborn
    - TensorFlow, Keras, Scikit-Learn
    - yFinance, ta-lib

**Dataset Choice: Yahoo Finance API via yfinance**
Why this dataset?

    - Free & accessible
    - Real-world stock data with actual issues (missing values, weekends/holdiays gaps, stock splits)
    - Forces you to handle non-stationarity, noise, and data irregularities
    - Contains OHLCV data needed for technical indicators

First, we're going to collect data. We're going to download 10 years worth of data for multiple stocks, generally the ones in FAANG (AAPL, MSFT, GOOGL, AMZN, TSLA), however for training we're going to focus on one stock initially

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
end_date = datetime.now()
start_date = end_date - timedelta(days=365*10)

data = {}
for ticker in tickers:
    df = yf.download(ticker, start=start_date, end= end_date, interval='1d')
    data[ticker] = df
    df.to_csv(f'data/raw_{ticker}.csv')


/var/folders/11/hr72w2z52lg8rt9z9977f9vh0000gn/T/ipykernel_80952/1737058979.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end= end_date, interval='1d')
[*********************100%***********************]  1 of 1 completed
/var/folders/11/hr72w2z52lg8rt9z9977f9vh0000gn/T/ipykernel_80952/1737058979.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end= end_date, interval='1d')
[*********************100%***********************]  1 of 1 completed
/var/folders/11/hr72w2z52lg8rt9z9977f9vh0000gn/T/ipykernel_80952/1737058979.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end= end_date, interval='1d')
[*********************100%***********************]  1 of 1 completed
/var/folders/11/hr72w2z52lg8rt9z9977f9vh0000gn/T/ipykernel_80952/1737058979.py:11: Fu

We're going to initially focus on the APPL stock

In [8]:
apple_data = data['AAPL']
apple_data.to_csv('data/raw_AAPL.csv')

Now, we move on to data cleaning and exploration

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Initial inspection

In [12]:
apple_data = pd.read_csv('data/raw_AAPL.csv', index_col=0, parse_dates=True)
# Flatten the multi-level columns if they exist
if isinstance(apple_data.columns, pd.MultiIndex):
    apple_data.columns = apple_data.columns.get_level_values(0)

print("First 5 rows of the dataset:")
print(apple_data.head())
print("---------------------")
print("Data Info:")
print(apple_data.info())
print("---------------------")
print("Statistical Summary:")
print(apple_data.describe())
print("---------------------")
print(f"\nMissing Values:\n{apple_data.isnull().sum()}")
print("---------------------")
print(f"\nData shape: {apple_data.shape}")
print("---------------------")
print(f"\nDate range:\nFrom {apple_data.index.min()} to {apple_data.index.max()}")

First 5 rows of the dataset:
                         Close                High                 Low  \
Price                                                                    
Ticker                    AAPL                AAPL                AAPL   
Date                       NaN                 NaN                 NaN   
2015-11-27   26.56248664855957   26.69776934479901  26.515138392952146   
2015-11-30  26.672971725463867  26.923242343649747  26.548963141609168   
2015-12-01   26.45652198791504   26.78796157229856  26.348297870682458   

                          Open     Volume  
Price                                      
Ticker                    AAPL       AAPL  
Date                       NaN        NaN  
2015-11-27  26.670712461512935   52185600  
2015-11-30   26.60307519629522  156721200  
2015-12-01   26.77443398769232  139409600  
---------------------
Data Info:
<class 'pandas.core.frame.DataFrame'>
Index: 2514 entries, Ticker to 2025-11-21
Data columns (total 5 columns):

/var/folders/11/hr72w2z52lg8rt9z9977f9vh0000gn/T/ipykernel_80952/131530973.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  apple_data = pd.read_csv('data/raw_AAPL.csv', index_col=0, parse_dates=True)
